In [1]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
from scipy.spatial.distance import squareform, pdist

In [2]:
# 데이터 파일 경로
data_path = './data.csv'  # 참가자가 제공받은 경로로 설정
data = pd.read_csv(data_path)

print("데이터 로드 완료!")
print(f"총 데이터 포인트 수: {len(data)}")
print(data.head())  # 데이터 확인

데이터 로드 완료!
총 데이터 포인트 수: 76
  point_id   x   y  demand
0    DEPOT   0   0       0
1  TOWN_01  34  94       2
2  TOWN_02  40  89       4
3  TOWN_03  39  95       1
4  TOWN_04  47  94       4


# OR-Tools

!pip install --upgrade ortools

## 최고점 재현

In [24]:
import math
import pandas as pd
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

def read_data(csv_path: str):
    """데이터를 읽고 전처리"""
    data_df = pd.read_csv(csv_path)
    data_df = data_df.sort_values(by='point_id').reset_index(drop=True)
    
    # DEPOT과 마을 데이터 분리
    depot_data = data_df[data_df['point_id'] == 'DEPOT'].iloc[0]
    towns_data = data_df[data_df['point_id'] != 'DEPOT'].reset_index(drop=True)
    
    # OR-Tools 데이터 준비
    locations = [(depot_data['x'], depot_data['y'])] + list(zip(towns_data['x'], towns_data['y']))
    demands = [0] + list(towns_data['demand'])
    
    return data_df, depot_data, towns_data, locations, demands

def create_data_model(locations, demands):
    """OR-Tools 데이터 모델 생성"""
    data = {}
    data['distance_matrix'] = [
        [int(round(math.sqrt((x1[0] - x2[0])**2 + (x1[1] - x2[1])**2) * 1000))
         for x2 in locations]
        for x1 in locations
    ]
    data['demands'] = demands
    data['vehicle_capacities'] = [25] * 75  # 최대 75대, 각 용량 25
    data['num_vehicles'] = 75
    data['depot'] = 0
    
    return data

def solve_vrp(data, time_limit_seconds=3600):
    """VRP 해결"""
    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']),
        data['num_vehicles'],
        data['depot']
    )
    routing = pywrapcp.RoutingModel(manager)
    
    # 거리 콜백
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    # 수요 콜백
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]
    
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    
    # 용량 제약 추가
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # slack
        data['vehicle_capacities'],
        True,  # start cumul to zero
        'Capacity'
    )
    
    # 탐색 파라미터 설정
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.seconds = time_limit_seconds
    search_parameters.log_search = True
    
    # 해 찾기
    solution = routing.SolveWithParameters(search_parameters)
    
    if not solution:
        return None, None
    
    # 결과 추출
    routes = []
    total_distance = 0
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route = []
        route_distance = 0
        
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(node)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            if not routing.IsEnd(index):
                route_distance += data['distance_matrix'][node][manager.IndexToNode(index)] / 1000
        
        if len(route) > 1:  # 실제로 마을을 방문한 경우만
            route.append(0)  # DEPOT으로 복귀
            routes.append(route)
            total_distance += route_distance
    
    return routes, total_distance

def convert_routes_to_submission(routes, towns_data):
    """경로를 제출 형식으로 변환"""
    total_route_town_ids = []
    
    for route in routes:
        route_town_ids = []
        for node_idx in route:
            if node_idx == 0:
                route_town_ids.append('DEPOT')
            else:
                town_id = towns_data.iloc[node_idx-1]['point_id']
                route_town_ids.append(town_id)
        total_route_town_ids.extend(route_town_ids[:-1])  # 마지막 DEPOT 제외
    
    total_route_town_ids.append('DEPOT')  # 최종 DEPOT 추가
    return total_route_town_ids

def main():
    try:
        # 데이터 읽기
        data_df, depot_data, towns_data, locations, demands = read_data('data.csv')
        
        # 데이터 모델 생성
        data = create_data_model(locations, demands)
        
        # VRP 해결
        routes, total_distance = solve_vrp(data)
        
        if routes:
            print("\n=== 산타의 배달 경로 ===")
            
            # 경로 출력
            for i, route in enumerate(routes, 1):
                route_town_ids = []
                for node_idx in route:
                    if node_idx == 0:
                        route_town_ids.append('DEPOT')
                    else:
                        town_id = towns_data.iloc[node_idx-1]['point_id']
                        route_town_ids.append(town_id)
                print(f"Route #{i}: {' -> '.join(route_town_ids)}")
            
            print(f"\n총 이동 거리: {total_distance:.2f} km")
            
            # 제출 파일 생성
            total_route_town_ids = convert_routes_to_submission(routes, towns_data)
            result_df = pd.DataFrame({'point_id': total_route_town_ids})
            result_df.to_csv('submission_or_imp.csv', index=False)
            print("\n[INFO] 제출 파일이 생성되었습니다: submission.csv")
            
        else:
            print("해를 찾을 수 없습니다!")
            
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


=== 산타의 배달 경로 ===
Route #1: DEPOT -> TOWN_32 -> TOWN_53 -> TOWN_66 -> TOWN_72 -> TOWN_08 -> TOWN_10 -> TOWN_38 -> DEPOT
Route #2: DEPOT -> TOWN_71 -> TOWN_23 -> TOWN_43 -> TOWN_44 -> TOWN_64 -> DEPOT
Route #3: DEPOT -> TOWN_19 -> TOWN_21 -> TOWN_55 -> TOWN_67 -> TOWN_60 -> TOWN_57 -> TOWN_69 -> DEPOT
Route #4: DEPOT -> TOWN_30 -> TOWN_22 -> TOWN_20 -> TOWN_24 -> TOWN_31 -> DEPOT
Route #5: DEPOT -> TOWN_70 -> TOWN_59 -> TOWN_51 -> TOWN_52 -> TOWN_75 -> TOWN_25 -> TOWN_49 -> TOWN_28 -> DEPOT
Route #6: DEPOT -> TOWN_34 -> TOWN_37 -> TOWN_46 -> TOWN_48 -> TOWN_54 -> TOWN_62 -> DEPOT
Route #7: DEPOT -> TOWN_04 -> TOWN_18 -> TOWN_74 -> TOWN_05 -> TOWN_01 -> TOWN_02 -> TOWN_03 -> TOWN_35 -> DEPOT
Route #8: DEPOT -> TOWN_61 -> TOWN_42 -> TOWN_13 -> TOWN_07 -> TOWN_09 -> TOWN_36 -> TOWN_40 -> TOWN_73 -> DEPOT
Route #9: DEPOT -> TOWN_65 -> TOWN_27 -> TOWN_63 -> TOWN_26 -> TOWN_45 -> DEPOT
Route #10: DEPOT -> TOWN_50 -> TOWN_56 -> TOWN_39 -> TOWN_41 -> TOWN_68 -> DEPOT
Route #11: DEPOT -> TOWN_2

In [166]:
import math
import pandas as pd

def calculate_route_distance(submission_path: str, data_path: str) -> float:
    """
    제출 파일의 경로에 대한 총 이동 거리 계산
    
    Args:
        submission_path: 제출 파일 경로 ('point_id' 컬럼을 가진 CSV)
        data_path: 원본 데이터 파일 경로 ('point_id', 'x', 'y' 컬럼을 가진 CSV)
        
    Returns:
        float: 총 이동 거리
    """
    # 데이터 로드
    submission_df = pd.read_csv(submission_path)
    data_df = pd.read_csv(data_path)
    
    # point_id를 기준으로 좌표 정보 병합
    route_df = pd.merge(
        submission_df,
        data_df[['point_id', 'x', 'y']],
        on='point_id',
        how='left'
    )
    
    # 총 거리 계산
    total_distance = 0
    for i in range(len(route_df) - 1):
        # 현재 지점과 다음 지점의 좌표
        current = route_df.iloc[i]
        next_point = route_df.iloc[i + 1]
        
        # 유클리드 거리 계산
        distance = math.sqrt(
            (current['x'] - next_point['x'])**2 + 
            (current['y'] - next_point['y'])**2
        )
        total_distance += distance
        
        # 이동 정보 출력
        print(f"{current['point_id']} -> {next_point['point_id']}: {distance:.4f}")
    
    return total_distance

def main():
    # 파일 경로
    submission_path = 'submission_or_imp.csv'  # 제출 파일
    data_path = 'data.csv'  # 원본 데이터
    
    try:
        # 총 거리 계산
        total_distance = calculate_route_distance(submission_path, data_path)
        
        print(f"\n=== 경로 거리 계산 결과 ===")
        print(f"총 이동 거리: {total_distance:.4f}")
        
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

DEPOT -> TOWN_32: 33.3017
TOWN_32 -> TOWN_19: 69.8140
TOWN_19 -> TOWN_57: 5.0000
TOWN_57 -> TOWN_69: 4.1231
TOWN_69 -> TOWN_55: 1.0000
TOWN_55 -> TOWN_67: 14.1421
TOWN_67 -> TOWN_60: 5.6569
TOWN_60 -> DEPOT: 98.7927
DEPOT -> TOWN_40: 47.1275
TOWN_40 -> TOWN_70: 49.0918
TOWN_70 -> TOWN_59: 8.0623
TOWN_59 -> TOWN_52: 4.0000
TOWN_52 -> TOWN_75: 11.0454
TOWN_75 -> TOWN_25: 10.0499
TOWN_25 -> TOWN_28: 7.2801
TOWN_28 -> TOWN_49: 27.7308
TOWN_49 -> DEPOT: 60.6712
DEPOT -> TOWN_35: 64.8460
TOWN_35 -> TOWN_01: 36.3456
TOWN_01 -> TOWN_05: 6.0000
TOWN_05 -> TOWN_74: 7.2111
TOWN_74 -> TOWN_03: 1.4142
TOWN_03 -> TOWN_04: 8.0623
TOWN_04 -> TOWN_02: 8.6023
TOWN_02 -> TOWN_33: 34.7131
TOWN_33 -> TOWN_37: 24.1661
TOWN_37 -> DEPOT: 40.2244
DEPOT -> TOWN_22: 91.0055
TOWN_22 -> TOWN_20: 10.6301
TOWN_20 -> TOWN_23: 5.0990
TOWN_23 -> TOWN_43: 3.0000
TOWN_43 -> TOWN_64: 2.8284
TOWN_64 -> TOWN_44: 9.0000
TOWN_44 -> DEPOT: 101.6514
DEPOT -> TOWN_16: 93.0054
TOWN_16 -> TOWN_24: 7.0000
TOWN_24 -> TOWN_21: 1.0000

## 최고점 시간 +

In [5]:
import math
import pandas as pd
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

def read_data(csv_path: str):
    """데이터를 읽고 전처리"""
    data_df = pd.read_csv(csv_path)
    data_df = data_df.sort_values(by='point_id').reset_index(drop=True)
    
    # DEPOT과 마을 데이터 분리
    depot_data = data_df[data_df['point_id'] == 'DEPOT'].iloc[0]
    towns_data = data_df[data_df['point_id'] != 'DEPOT'].reset_index(drop=True)
    
    # OR-Tools 데이터 준비
    locations = [(depot_data['x'], depot_data['y'])] + list(zip(towns_data['x'], towns_data['y']))
    demands = [0] + list(towns_data['demand'])
    
    return data_df, depot_data, towns_data, locations, demands

def create_data_model(locations, demands):
    """OR-Tools 데이터 모델 생성"""
    data = {}
    data['distance_matrix'] = [
        [int(math.sqrt((x1[0] - x2[0])**2 + (x1[1] - x2[1])**2) * 1000)
         for x2 in locations]
        for x1 in locations
    ]
    data['demands'] = demands
    data['vehicle_capacities'] = [25] * 75  # 최대 75대, 각 용량 25
    data['num_vehicles'] = 75
    data['depot'] = 0
    
    return data

def solve_vrp(data, time_limit_seconds=7200):
    """VRP 해결"""
    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']),
        data['num_vehicles'],
        data['depot']
    )
    routing = pywrapcp.RoutingModel(manager)
    
    # 거리 콜백
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    # 수요 콜백
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]
    
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    
    # 용량 제약 추가
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # slack
        data['vehicle_capacities'],
        True,  # start cumul to zero
        'Capacity'
    )
    
    # 탐색 파라미터 설정
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.seconds = time_limit_seconds
    search_parameters.log_search = True
    
    # 해 찾기
    solution = routing.SolveWithParameters(search_parameters)
    
    if not solution:
        return None, None
    
    # 결과 추출
    routes = []
    total_distance = 0
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route = []
        route_distance = 0
        
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(node)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            if not routing.IsEnd(index):
                route_distance += data['distance_matrix'][node][manager.IndexToNode(index)] / 1000
        
        if len(route) > 1:  # 실제로 마을을 방문한 경우만
            route.append(0)  # DEPOT으로 복귀
            routes.append(route)
            total_distance += route_distance
    
    return routes, total_distance

def convert_routes_to_submission(routes, towns_data):
    """경로를 제출 형식으로 변환"""
    total_route_town_ids = []
    
    for route in routes:
        route_town_ids = []
        for node_idx in route:
            if node_idx == 0:
                route_town_ids.append('DEPOT')
            else:
                town_id = towns_data.iloc[node_idx-1]['point_id']
                route_town_ids.append(town_id)
        total_route_town_ids.extend(route_town_ids[:-1])  # 마지막 DEPOT 제외
    
    total_route_town_ids.append('DEPOT')  # 최종 DEPOT 추가
    return total_route_town_ids

def main():
    try:
        # 데이터 읽기
        data_df, depot_data, towns_data, locations, demands = read_data('data.csv')
        
        # 데이터 모델 생성
        data = create_data_model(locations, demands)
        
        # VRP 해결
        routes, total_distance = solve_vrp(data)
        
        if routes:
            print("\n=== 산타의 배달 경로 ===")
            
            # 경로 출력
            for i, route in enumerate(routes, 1):
                route_town_ids = []
                for node_idx in route:
                    if node_idx == 0:
                        route_town_ids.append('DEPOT')
                    else:
                        town_id = towns_data.iloc[node_idx-1]['point_id']
                        route_town_ids.append(town_id)
                print(f"Route #{i}: {' -> '.join(route_town_ids)}")
            
            print(f"\n총 이동 거리: {total_distance:.2f} km")
            
            # 제출 파일 생성
            total_route_town_ids = convert_routes_to_submission(routes, towns_data)
            result_df = pd.DataFrame({'point_id': total_route_town_ids})
            result_df.to_csv('submission_or_imp_.csv', index=False)
            print("\n[INFO] 제출 파일이 생성되었습니다: submission_or_imp_.csv")
            
        else:
            print("해를 찾을 수 없습니다!")
            
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


=== 산타의 배달 경로 ===
Route #1: DEPOT -> TOWN_32 -> TOWN_19 -> TOWN_57 -> TOWN_69 -> TOWN_55 -> TOWN_67 -> TOWN_60 -> DEPOT
Route #2: DEPOT -> TOWN_40 -> TOWN_70 -> TOWN_59 -> TOWN_52 -> TOWN_75 -> TOWN_25 -> TOWN_28 -> TOWN_49 -> DEPOT
Route #3: DEPOT -> TOWN_35 -> TOWN_01 -> TOWN_05 -> TOWN_74 -> TOWN_03 -> TOWN_04 -> TOWN_02 -> TOWN_33 -> TOWN_37 -> DEPOT
Route #4: DEPOT -> TOWN_22 -> TOWN_20 -> TOWN_23 -> TOWN_43 -> TOWN_64 -> TOWN_44 -> DEPOT
Route #5: DEPOT -> TOWN_16 -> TOWN_24 -> TOWN_21 -> TOWN_14 -> TOWN_15 -> DEPOT
Route #6: DEPOT -> TOWN_08 -> TOWN_53 -> TOWN_71 -> TOWN_66 -> TOWN_72 -> TOWN_51 -> TOWN_36 -> TOWN_58 -> DEPOT
Route #7: DEPOT -> TOWN_65 -> TOWN_63 -> TOWN_27 -> TOWN_26 -> TOWN_45 -> DEPOT
Route #8: DEPOT -> TOWN_34 -> TOWN_48 -> TOWN_18 -> TOWN_46 -> TOWN_62 -> TOWN_54 -> DEPOT
Route #9: DEPOT -> TOWN_56 -> TOWN_38 -> TOWN_07 -> TOWN_47 -> TOWN_73 -> TOWN_06 -> TOWN_09 -> TOWN_42 -> TOWN_61 -> TOWN_17 -> DEPOT
Route #10: DEPOT -> TOWN_29 -> TOWN_13 -> TOWN_50 ->

In [6]:
import math
import pandas as pd

def calculate_route_distance(submission_path: str, data_path: str) -> float:
    """
    제출 파일의 경로에 대한 총 이동 거리 계산
    
    Args:
        submission_path: 제출 파일 경로 ('point_id' 컬럼을 가진 CSV)
        data_path: 원본 데이터 파일 경로 ('point_id', 'x', 'y' 컬럼을 가진 CSV)
        
    Returns:
        float: 총 이동 거리
    """
    # 데이터 로드
    submission_df = pd.read_csv(submission_path)
    data_df = pd.read_csv(data_path)
    
    # point_id를 기준으로 좌표 정보 병합
    route_df = pd.merge(
        submission_df,
        data_df[['point_id', 'x', 'y']],
        on='point_id',
        how='left'
    )
    
    # 총 거리 계산
    total_distance = 0
    for i in range(len(route_df) - 1):
        # 현재 지점과 다음 지점의 좌표
        current = route_df.iloc[i]
        next_point = route_df.iloc[i + 1]
        
        # 유클리드 거리 계산
        distance = math.sqrt(
            (current['x'] - next_point['x'])**2 + 
            (current['y'] - next_point['y'])**2
        )
        total_distance += distance
        
        # 이동 정보 출력
        print(f"{current['point_id']} -> {next_point['point_id']}: {distance:.4f}")
    
    return total_distance

def main():
    # 파일 경로
    submission_path = 'submission_or_imp_.csv'  # 제출 파일
    data_path = 'data.csv'  # 원본 데이터
    
    try:
        # 총 거리 계산
        total_distance = calculate_route_distance(submission_path, data_path)
        
        print(f"\n=== 경로 거리 계산 결과 ===")
        print(f"총 이동 거리: {total_distance:.4f}")
        
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

DEPOT -> TOWN_32: 33.3017
TOWN_32 -> TOWN_19: 69.8140
TOWN_19 -> TOWN_57: 5.0000
TOWN_57 -> TOWN_69: 4.1231
TOWN_69 -> TOWN_55: 1.0000
TOWN_55 -> TOWN_67: 14.1421
TOWN_67 -> TOWN_60: 5.6569
TOWN_60 -> DEPOT: 98.7927
DEPOT -> TOWN_40: 47.1275
TOWN_40 -> TOWN_70: 49.0918
TOWN_70 -> TOWN_59: 8.0623
TOWN_59 -> TOWN_52: 4.0000
TOWN_52 -> TOWN_75: 11.0454
TOWN_75 -> TOWN_25: 10.0499
TOWN_25 -> TOWN_28: 7.2801
TOWN_28 -> TOWN_49: 27.7308
TOWN_49 -> DEPOT: 60.6712
DEPOT -> TOWN_35: 64.8460
TOWN_35 -> TOWN_01: 36.3456
TOWN_01 -> TOWN_05: 6.0000
TOWN_05 -> TOWN_74: 7.2111
TOWN_74 -> TOWN_03: 1.4142
TOWN_03 -> TOWN_04: 8.0623
TOWN_04 -> TOWN_02: 8.6023
TOWN_02 -> TOWN_33: 34.7131
TOWN_33 -> TOWN_37: 24.1661
TOWN_37 -> DEPOT: 40.2244
DEPOT -> TOWN_22: 91.0055
TOWN_22 -> TOWN_20: 10.6301
TOWN_20 -> TOWN_23: 5.0990
TOWN_23 -> TOWN_43: 3.0000
TOWN_43 -> TOWN_64: 2.8284
TOWN_64 -> TOWN_44: 9.0000
TOWN_44 -> DEPOT: 101.6514
DEPOT -> TOWN_16: 93.0054
TOWN_16 -> TOWN_24: 7.0000
TOWN_24 -> TOWN_21: 1.0000

## 파라미터 조정

In [ ]:
import math
import pandas as pd
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

def read_data(csv_path: str):
    """데이터를 읽고 전처리"""
    data_df = pd.read_csv(csv_path)
    data_df = data_df.sort_values(by='point_id').reset_index(drop=True)
    
    # DEPOT과 마을 데이터 분리
    depot_data = data_df[data_df['point_id'] == 'DEPOT'].iloc[0]
    towns_data = data_df[data_df['point_id'] != 'DEPOT'].reset_index(drop=True)
    
    # OR-Tools 데이터 준비
    locations = [(depot_data['x'], depot_data['y'])] + list(zip(towns_data['x'], towns_data['y']))
    demands = [0] + list(towns_data['demand'])
    
    return data_df, depot_data, towns_data, locations, demands

def create_data_model(locations, demands):
    """OR-Tools 데이터 모델 생성"""
    data = {}
    # 정수 스케일을 유지하되, 조금 더 세밀하게(예: 1km -> 10000 스케일) 해볼 수도 있음
    # 혹은 float 대신 아래처럼 int를 쓰되, 소수점 자르는 방식을 round로 변경 가능
    data['distance_matrix'] = [
        [int(round(math.sqrt((x1[0] - x2[0])**2 + (x1[1] - x2[1])**2)*1000))
         for x2 in locations]
        for x1 in locations
    ]
    
    data['demands'] = demands
    data['vehicle_capacities'] = [25] * 75  # 최대 75대, 각 용량 25
    data['num_vehicles'] = 75
    data['depot'] = 0
    
    return data

def solve_vrp(data, time_limit_seconds=3600):
    """VRP 해결(개선판)"""
    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']),
        data['num_vehicles'],
        data['depot']
    )
    routing = pywrapcp.RoutingModel(manager)
    
    # 거리 콜백
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    # 수요 콜백
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]
    
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    
    # 용량 제약 추가
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # slack
        data['vehicle_capacities'],
        True,  # start cumul to zero
        'Capacity'
    )
    
    # 탐색 파라미터 설정
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    
    # === 초기 해법 전략 ===
    # 적절한 초기해법을 찾는 전략: PATH_CHEAPEST_ARC, SAVINGS, SWEEP, CHRISTOFIDES 등 다양
    # 문제에 따라 다양한 설정을 테스트해 볼 수 있음
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )
    
    # === 메타 휴리스틱(로컬 서치) 설정 ===
    # GUIDED_LOCAL_SEARCH, SIMULATED_ANNEALING, TABU_SEARCH 등 변경 가능
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    
    # === 로컬 서치 오퍼레이터 강화 ===
    # 필요에 따라 활성화 가능 (성능과 시간 트레이드오프)
    search_parameters.local_search_operators.use_relocate = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_exchange = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_cross = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_lin_kernighan = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_tsp_opt = pywrapcp.BOOL_TRUE
    
    # === LNS 계열 기법 ===
    # 경로단위 LNS, TSP LNS, 비활성화 LNS 등 가능
    search_parameters.local_search_operators.use_path_lns = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_tsp_lns = pywrapcp.BOOL_TRUE
    # 아래도 시도해볼 수 있음
    # search_parameters.local_search_operators.use_full_path_lns = pywrapcp.BOOL_TRUE
    
    # === 시간 제한 ===
    search_parameters.time_limit.seconds = time_limit_seconds
    search_parameters.log_search = True  # 탐색과정 로그 표시
    
    # === Guided Local Search 파라미터 튜닝 ===
    # 페널티 가중치(람다 계수). 크면 페널티가 더 세게 부여되어, 경로가 다양해짐
    search_parameters.guided_local_search_lambda_coefficient = 0.1
    
    # === 다중 스레드 병렬화(선택) ===
    # (병렬화 시 결과 재현성 보장이 안 될 수 있음)
    # search_parameters.number_of_solutions_to_collect = 1
    # search_parameters.solution_limit = 1000000000
    # search_parameters.use_cp = True  # OR-Tools 8.x 이후 버전에서만 작동
    
    # 해 찾기
    solution = routing.SolveWithParameters(search_parameters)
    
    if not solution:
        return None, None
    
    # 결과 추출
    routes = []
    total_distance = 0.0
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route = []
        route_distance = 0
        
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(node)
            prev_index = index
            index = solution.Value(routing.NextVar(index))
            if not routing.IsEnd(index):
                next_node = manager.IndexToNode(index)
                # 다시 실제 km 단위로 환산
                route_distance += data['distance_matrix'][node][next_node] / 1000.0
        
        # 실제로 마을을 방문한 경우만 기록 (DEPOT만 방문하고 끝나는 경우 제외)
        if len(route) > 1:
            # DEPOT 복귀
            route.append(0)
            routes.append(route)
            total_distance += route_distance
    
    return routes, total_distance

def convert_routes_to_submission(routes, towns_data):
    """경로를 제출 형식으로 변환"""
    total_route_town_ids = []
    
    for route in routes:
        route_town_ids = []
        for node_idx in route:
            if node_idx == 0:
                route_town_ids.append('DEPOT')
            else:
                town_id = towns_data.iloc[node_idx-1]['point_id']
                route_town_ids.append(town_id)
        # 마지막 DEPOT 제외하고 다음 경로로 넘어감
        total_route_town_ids.extend(route_town_ids[:-1])
    
    # 최종 DEPOT 추가
    total_route_town_ids.append('DEPOT')
    return total_route_town_ids

def main():
    try:
        # 데이터 읽기
        data_df, depot_data, towns_data, locations, demands = read_data('data.csv')
        
        # 데이터 모델 생성
        data = create_data_model(locations, demands)
        
        # VRP 해결
        # 시간 제한을 늘려서 더 깊이 탐색해볼 수 있음 (예: 10800초 = 3시간 등)
        routes, total_distance = solve_vrp(data, time_limit_seconds=7200)
        
        if routes:
            print("\n=== 산타의 배달 경로 ===")
            for i, route in enumerate(routes, 1):
                route_town_ids = []
                for node_idx in route:
                    if node_idx == 0:
                        route_town_ids.append('DEPOT')
                    else:
                        town_id = towns_data.iloc[node_idx-1]['point_id']
                        route_town_ids.append(town_id)
                print(f"Route #{i}: {' -> '.join(route_town_ids)}")
            
            print(f"\n총 이동 거리(환산): {total_distance:.2f} km")
            
            # 제출 파일 생성
            total_route_town_ids = convert_routes_to_submission(routes, towns_data)
            result_df = pd.DataFrame({'point_id': total_route_town_ids})
            result_df.to_csv('submission_or_imp_2.csv', index=False)
            print("\n[INFO] 제출 파일이 생성되었습니다: submission_or_imp_2.csv")
            
        else:
            print("해를 찾을 수 없습니다!")
            
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


## 파리미터 조정 2

In [7]:
import math
import random
import pandas as pd
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

def read_data(csv_path: str):
    """데이터를 읽고 전처리"""
    data_df = pd.read_csv(csv_path)
    data_df = data_df.sort_values(by='point_id').reset_index(drop=True)
    
    # DEPOT과 마을 데이터 분리
    depot_data = data_df[data_df['point_id'] == 'DEPOT'].iloc[0]
    towns_data = data_df[data_df['point_id'] != 'DEPOT'].reset_index(drop=True)
    
    # OR-Tools 데이터 준비
    locations = [(depot_data['x'], depot_data['y'])] + list(zip(towns_data['x'], towns_data['y']))
    demands = [0] + list(towns_data['demand'])
    
    return data_df, depot_data, towns_data, locations, demands

def create_data_model(locations, demands):
    """OR-Tools 데이터 모델 생성"""
    data = {}
    # 정수 스케일 (m단위 혹은 0.001km 단위)
    data['distance_matrix'] = [
        [int(round(math.sqrt((x1[0] - x2[0])**2 + (x1[1] - x2[1])**2) * 1000))
         for x2 in locations]
        for x1 in locations
    ]
    data['demands'] = demands
    data['vehicle_capacities'] = [25] * 75  # 최대 75대, 각 용량 25
    data['num_vehicles'] = 75
    data['depot'] = 0
    return data

def solve_vrp_single_run(data, 
                        time_limit_seconds=300,
                        first_sol_strategy=routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC,
                        metaheuristic=routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH,
                        random_seed=0):
    """
    하나의 설정(초기 해법, 메타휴리스틱, 랜덤시드 등)에 대해 VRP를 풀고 결과를 반환.
    로컬 서치 오퍼레이터를 적극적으로 사용.
    """
    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']),
        data['num_vehicles'],
        data['depot']
    )
    routing = pywrapcp.RoutingModel(manager)
    
    # === 콜백 등록 ===
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    distance_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(distance_callback_index)
    
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]
    
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    
    # === 용량 제약 ===
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # slack
        data['vehicle_capacities'],
        True,
        'Capacity'
    )
    
    # === 탐색 파라미터 설정 ===
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    
    # 1) 초기 해법 전략
    search_parameters.first_solution_strategy = first_sol_strategy
    
    # 2) 메타휴리스틱
    search_parameters.local_search_metaheuristic = metaheuristic
    
    # 3) 로컬 서치 오퍼레이터 적극 활용
    # (필요 없거나 너무 느리면 주석 처리)
    search_parameters.local_search_operators.use_relocate = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_relocate_neighbors = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_exchange = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_cross = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_lin_kernighan = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_tsp_opt = pywrapcp.BOOL_TRUE
    
    search_parameters.local_search_operators.use_make_active = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_make_inactive = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_make_chain_inactive = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_swap_active = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_extended_swap_active = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_node_pair_swap_active = pywrapcp.BOOL_TRUE
    
    # LNS 계열 기법
    search_parameters.local_search_operators.use_path_lns = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_full_path_lns = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_tsp_lns = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_inactive_lns = pywrapcp.BOOL_TRUE
    
    # 시간 제한
    search_parameters.time_limit.seconds = time_limit_seconds
    search_parameters.log_search = False  # 상세 탐색 로그를 보고 싶다면 True
    
    # GLS 파라미터 등
    search_parameters.guided_local_search_lambda_coefficient = 0.1
    
    # 4) 랜덤 시드
    # if random_seed != 0:
    #     search_parameters.random_seed = random_seed
    
    # 해 찾기
    solution = routing.SolveWithParameters(search_parameters)
    if not solution:
        return None, None
    
    # === 결과 파싱 ===
    routes = []
    total_distance = 0.0
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route = []
        route_distance = 0
        
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(node)
            prev_index = index
            index = solution.Value(routing.NextVar(index))
            if not routing.IsEnd(index):
                next_node = manager.IndexToNode(index)
                route_distance += data['distance_matrix'][node][next_node] / 1000.0
        
        if len(route) > 1:
            # DEPOT 복귀
            route.append(0)
            routes.append(route)
            total_distance += route_distance
    
    return routes, total_distance

def solve_vrp_multi_start(data, 
                          time_limit_seconds=300,
                          num_runs=5):
    """
    4) 멀티 스타트 기법 적용:
    여러 번 다른 설정(랜덤 시드, 초기 해법, 메타휴리스틱)으로 실행 후,
    가장 좋은 해를 반환.
    """
    
    # 시도해 볼 초기 해법(FirstSolutionStrategy) 모음
    possible_first_strategies = [
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC,
        routing_enums_pb2.FirstSolutionStrategy.SAVINGS,
        routing_enums_pb2.FirstSolutionStrategy.SWEEP,
        routing_enums_pb2.FirstSolutionStrategy.CHRISTOFIDES,
    ]
    
    # 시도해 볼 메타휴리스틱(LNS/TABU/SA/GLS) 모음
    possible_metaheuristics = [
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH,
        routing_enums_pb2.LocalSearchMetaheuristic.SIMULATED_ANNEALING,
        routing_enums_pb2.LocalSearchMetaheuristic.TABU_SEARCH
    ]
    
    best_routes = None
    best_distance = float('inf')
    
    for i in range(num_runs):
        # 무작위로 하나씩 골라본다 (혹은 순회)
        fs = random.choice(possible_first_strategies)
        mh = random.choice(possible_metaheuristics)
        seed = random.randint(1, 99999)
        
        print(f"[INFO] Multi-Start Run #{i+1}: FS={fs}, MH={mh}, Seed={seed}")
        
        # 단일 실행
        routes, total_dist = solve_vrp_single_run(
            data,
            time_limit_seconds=time_limit_seconds,
            first_sol_strategy=fs,
            metaheuristic=mh,
            random_seed=seed
        )
        
        # 해 검증
        if routes and total_dist < best_distance:
            best_distance = total_dist
            best_routes = routes
    
    if best_routes:
        print(f"[INFO] Best Distance from {num_runs} runs: {best_distance:.2f}")
    else:
        print("[WARN] No feasible solution found in multi-start.")
    
    return best_routes, best_distance

def convert_routes_to_submission(routes, towns_data):
    """경로를 제출 형식으로 변환"""
    total_route_town_ids = []
    
    for route in routes:
        route_town_ids = []
        for node_idx in route:
            if node_idx == 0:
                route_town_ids.append('DEPOT')
            else:
                town_id = towns_data.iloc[node_idx - 1]['point_id']
                route_town_ids.append(town_id)
        total_route_town_ids.extend(route_town_ids[:-1])  # 마지막 DEPOT 제외
    
    total_route_town_ids.append('DEPOT')  # 최종 DEPOT 추가
    return total_route_town_ids

def main():
    try:
        # 데이터 읽기
        data_df, depot_data, towns_data, locations, demands = read_data('data.csv')
        
        # 데이터 모델 생성
        data = create_data_model(locations, demands)
        
        # 4) 멀티 스타트: 여러 번 실행 -> 최고 해 선택
        # 시간은 예시로 300초씩(num_runs=5번) = 총 25분 정도
        routes, best_dist = solve_vrp_multi_start(
            data, 
            time_limit_seconds=300,
            num_runs=5
        )
        
        # 최적 해 출력/저장
        if routes:
            print("\n=== 산타의 배달 경로 ===")
            for i, route in enumerate(routes, 1):
                route_town_ids = []
                for node_idx in route:
                    if node_idx == 0:
                        route_town_ids.append('DEPOT')
                    else:
                        town_id = towns_data.iloc[node_idx-1]['point_id']
                        route_town_ids.append(town_id)
                print(f"Route #{i}: {' -> '.join(route_town_ids)}")
            
            print(f"\n총 이동 거리(환산): {best_dist:.2f} km")
            
            # 제출 파일 생성
            total_route_town_ids = convert_routes_to_submission(routes, towns_data)
            result_df = pd.DataFrame({'point_id': total_route_town_ids})
            result_df.to_csv('submission_multi_start.csv', index=False)
            print("\n[INFO] 제출 파일이 생성되었습니다: submission_multi_start.csv")
            
        else:
            print("[ERROR] 해를 찾을 수 없습니다!")
            
    except Exception as e:
        print(f"[ERROR] {e}")

if __name__ == "__main__":
    main()


[INFO] Multi-Start Run #1: FS=13, MH=4, Seed=88865
[INFO] Multi-Start Run #2: FS=3, MH=3, Seed=80027
[INFO] Multi-Start Run #3: FS=10, MH=4, Seed=42629
[INFO] Multi-Start Run #4: FS=3, MH=4, Seed=30319
[INFO] Multi-Start Run #5: FS=3, MH=3, Seed=57473
[INFO] Best Distance from 5 runs: 1426.81

=== 산타의 배달 경로 ===
Route #1: DEPOT -> TOWN_46 -> TOWN_02 -> TOWN_04 -> TOWN_67 -> TOWN_60 -> TOWN_17 -> DEPOT
Route #2: DEPOT -> TOWN_08 -> TOWN_72 -> TOWN_66 -> TOWN_71 -> TOWN_53 -> TOWN_61 -> DEPOT
Route #3: DEPOT -> TOWN_25 -> TOWN_75 -> TOWN_52 -> TOWN_59 -> TOWN_70 -> TOWN_51 -> TOWN_47 -> TOWN_07 -> DEPOT
Route #4: DEPOT -> TOWN_43 -> TOWN_64 -> TOWN_44 -> TOWN_05 -> TOWN_03 -> TOWN_74 -> TOWN_01 -> DEPOT
Route #5: DEPOT -> TOWN_23 -> TOWN_14 -> TOWN_20 -> TOWN_21 -> TOWN_24 -> TOWN_22 -> DEPOT
Route #6: DEPOT -> TOWN_19 -> TOWN_55 -> TOWN_69 -> TOWN_57 -> TOWN_42 -> TOWN_09 -> TOWN_06 -> TOWN_73 -> DEPOT
Route #7: DEPOT -> TOWN_58 -> TOWN_40 -> TOWN_38 -> TOWN_34 -> TOWN_48 -> TOWN_18 -> T

In [9]:
import math
import pandas as pd

def calculate_route_distance(submission_path: str, data_path: str) -> float:
    """
    제출 파일의 경로에 대한 총 이동 거리 계산
    
    Args:
        submission_path: 제출 파일 경로 ('point_id' 컬럼을 가진 CSV)
        data_path: 원본 데이터 파일 경로 ('point_id', 'x', 'y' 컬럼을 가진 CSV)
        
    Returns:
        float: 총 이동 거리
    """
    # 데이터 로드
    submission_df = pd.read_csv(submission_path)
    data_df = pd.read_csv(data_path)
    
    # point_id를 기준으로 좌표 정보 병합
    route_df = pd.merge(
        submission_df,
        data_df[['point_id', 'x', 'y']],
        on='point_id',
        how='left'
    )
    
    # 총 거리 계산
    total_distance = 0
    for i in range(len(route_df) - 1):
        # 현재 지점과 다음 지점의 좌표
        current = route_df.iloc[i]
        next_point = route_df.iloc[i + 1]
        
        # 유클리드 거리 계산
        distance = math.sqrt(
            (current['x'] - next_point['x'])**2 + 
            (current['y'] - next_point['y'])**2
        )
        total_distance += distance
        
        # 이동 정보 출력
        print(f"{current['point_id']} -> {next_point['point_id']}: {distance:.4f}")
    
    return total_distance

def main():
    # 파일 경로
    submission_path = 'submission_multi_start.csv'  # 제출 파일
    data_path = 'data.csv'  # 원본 데이터
    
    try:
        # 총 거리 계산
        total_distance = calculate_route_distance(submission_path, data_path)
        
        print(f"\n=== 경로 거리 계산 결과 ===")
        print(f"총 이동 거리: {total_distance:.4f}")
        
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

DEPOT -> TOWN_46: 72.9452
TOWN_46 -> TOWN_02: 25.4951
TOWN_02 -> TOWN_04: 8.6023
TOWN_04 -> TOWN_67: 10.8167
TOWN_67 -> TOWN_60: 5.6569
TOWN_60 -> TOWN_17: 17.0000
TOWN_17 -> DEPOT: 91.4385
DEPOT -> TOWN_08: 109.0413
TOWN_08 -> TOWN_72: 14.3178
TOWN_72 -> TOWN_66: 18.9737
TOWN_66 -> TOWN_71: 16.2788
TOWN_71 -> TOWN_53: 8.6023
TOWN_53 -> TOWN_61: 15.2315
TOWN_61 -> DEPOT: 100.4988
DEPOT -> TOWN_25: 91.3947
TOWN_25 -> TOWN_75: 10.0499
TOWN_75 -> TOWN_52: 11.0454
TOWN_52 -> TOWN_59: 4.0000
TOWN_59 -> TOWN_70: 8.0623
TOWN_70 -> TOWN_51: 8.6023
TOWN_51 -> TOWN_47: 14.3178
TOWN_47 -> TOWN_07: 1.0000
TOWN_07 -> DEPOT: 86.0523
DEPOT -> TOWN_43: 101.2719
TOWN_43 -> TOWN_64: 2.8284
TOWN_64 -> TOWN_44: 9.0000
TOWN_44 -> TOWN_05: 7.2801
TOWN_05 -> TOWN_03: 7.0711
TOWN_03 -> TOWN_74: 1.4142
TOWN_74 -> TOWN_01: 4.0000
TOWN_01 -> DEPOT: 99.9600
DEPOT -> TOWN_23: 100.8415
TOWN_23 -> TOWN_14: 4.0000
TOWN_14 -> TOWN_20: 1.4142
TOWN_20 -> TOWN_21: 6.0828
TOWN_21 -> TOWN_24: 1.0000
TOWN_24 -> TOWN_22: 9.0

## 파라미터 조정 3

In [6]:
import math
import random
import pandas as pd
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

def read_data(csv_path: str):
    """데이터를 읽고 전처리"""
    data_df = pd.read_csv(csv_path)
    data_df = data_df.sort_values(by='point_id').reset_index(drop=True)
    
    # DEPOT과 마을 데이터 분리
    depot_data = data_df[data_df['point_id'] == 'DEPOT'].iloc[0]
    towns_data = data_df[data_df['point_id'] != 'DEPOT'].reset_index(drop=True)
    
    # OR-Tools 데이터 준비
    locations = [(depot_data['x'], depot_data['y'])] + list(zip(towns_data['x'], towns_data['y']))
    demands = [0] + list(towns_data['demand'])
    
    return data_df, depot_data, towns_data, locations, demands

def create_data_model(locations, demands):
    """OR-Tools 데이터 모델 생성"""
    data = {}
    # 정수 스케일 (m단위 혹은 0.001km 단위)
    data['distance_matrix'] = [
        [int(round(math.sqrt((x1[0] - x2[0])**2 + (x1[1] - x2[1])**2) * 100000))
         for x2 in locations]
        for x1 in locations
    ]
    data['demands'] = demands
    data['vehicle_capacities'] = [25] * 75  # 최대 75대, 각 용량 25
    data['num_vehicles'] = 75
    data['depot'] = 0
    return data

def solve_vrp_single_run(data, 
                        time_limit_seconds=600,
                        first_sol_strategy=routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC,
                        metaheuristic=routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH,
                        random_seed=0):
    """
    하나의 설정(초기 해법, 메타휴리스틱, 랜덤시드 등)에 대해 VRP를 풀고 결과를 반환.
    로컬 서치 오퍼레이터를 적극적으로 사용.
    """
    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']),
        data['num_vehicles'],
        data['depot']
    )
    routing = pywrapcp.RoutingModel(manager)
    
    # === 콜백 등록 ===
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    distance_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(distance_callback_index)
    
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]
    
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    
    # === 용량 제약 ===
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # slack
        data['vehicle_capacities'],
        True,
        'Capacity'
    )
    
    # === 탐색 파라미터 설정 ===
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    
    # 1) 초기 해법 전략
    search_parameters.first_solution_strategy = first_sol_strategy
    
    # 2) 메타휴리스틱
    search_parameters.local_search_metaheuristic = metaheuristic
    
    # 3) 로컬 서치 오퍼레이터 적극 활용
    # (필요 없거나 너무 느리면 주석 처리)
    search_parameters.local_search_operators.use_relocate = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_relocate_neighbors = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_exchange = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_cross = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_lin_kernighan = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_tsp_opt = pywrapcp.BOOL_TRUE
    
    search_parameters.local_search_operators.use_make_active = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_make_inactive = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_make_chain_inactive = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_swap_active = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_extended_swap_active = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_node_pair_swap_active = pywrapcp.BOOL_TRUE
    
    # LNS 계열 기법
    search_parameters.local_search_operators.use_path_lns = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_full_path_lns = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_tsp_lns = pywrapcp.BOOL_TRUE
    search_parameters.local_search_operators.use_inactive_lns = pywrapcp.BOOL_TRUE
    
    # 시간 제한
    search_parameters.time_limit.seconds = time_limit_seconds
    search_parameters.log_search = False  # 상세 탐색 로그를 보고 싶다면 True
    
    # GLS 파라미터 등
    search_parameters.guided_local_search_lambda_coefficient = 0.5
    
    # 4) 랜덤 시드
    # if random_seed != 0:
    #     search_parameters.random_seed = random_seed
    
    # 해 찾기
    solution = routing.SolveWithParameters(search_parameters)
    if not solution:
        return None, None
    
    # === 결과 파싱 ===
    routes = []
    total_distance = 0.0
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route = []
        route_distance = 0
        
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(node)
            prev_index = index
            index = solution.Value(routing.NextVar(index))
            if not routing.IsEnd(index):
                next_node = manager.IndexToNode(index)
                route_distance += data['distance_matrix'][node][next_node] / 1000.0
        
        if len(route) > 1:
            # DEPOT 복귀
            route.append(0)
            routes.append(route)
            total_distance += route_distance
    
    return routes, total_distance

def solve_vrp_multi_start(data, 
                          time_limit_seconds=600,
                          num_runs=10):
    """
    4) 멀티 스타트 기법 적용:
    여러 번 다른 설정(랜덤 시드, 초기 해법, 메타휴리스틱)으로 실행 후,
    가장 좋은 해를 반환.
    """
    
    # 시도해 볼 초기 해법(FirstSolutionStrategy) 모음
    possible_first_strategies = [
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC,
        routing_enums_pb2.FirstSolutionStrategy.SAVINGS,
        routing_enums_pb2.FirstSolutionStrategy.SWEEP,
        routing_enums_pb2.FirstSolutionStrategy.CHRISTOFIDES,
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION,  # 추가
        routing_enums_pb2.FirstSolutionStrategy.LOCAL_CHEAPEST_INSERTION     # 추가
    ]
    
    # 시도해 볼 메타휴리스틱(LNS/TABU/SA/GLS) 모음
    possible_metaheuristics = [
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH,
        routing_enums_pb2.LocalSearchMetaheuristic.SIMULATED_ANNEALING,
        routing_enums_pb2.LocalSearchMetaheuristic.TABU_SEARCH
    ]
    
    best_routes = None
    best_distance = float('inf')
    
    for i in range(num_runs):
        # 무작위로 하나씩 골라본다 (혹은 순회)
        fs = random.choice(possible_first_strategies)
        mh = random.choice(possible_metaheuristics)
        seed = random.randint(1, 99999)
        
        print(f"[INFO] Multi-Start Run #{i+1}: FS={fs}, MH={mh}, Seed={seed}")
        
        # 단일 실행
        routes, total_dist = solve_vrp_single_run(
            data,
            time_limit_seconds=time_limit_seconds,
            first_sol_strategy=fs,
            metaheuristic=mh,
            random_seed=seed
        )
        
        # 해 검증
        if routes and total_dist < best_distance:
            best_distance = total_dist
            best_routes = routes
    
    if best_routes:
        print(f"[INFO] Best Distance from {num_runs} runs: {best_distance:.2f}")
    else:
        print("[WARN] No feasible solution found in multi-start.")
    
    return best_routes, best_distance

def convert_routes_to_submission(routes, towns_data):
    """경로를 제출 형식으로 변환"""
    total_route_town_ids = []
    
    for route in routes:
        route_town_ids = []
        for node_idx in route:
            if node_idx == 0:
                route_town_ids.append('DEPOT')
            else:
                town_id = towns_data.iloc[node_idx - 1]['point_id']
                route_town_ids.append(town_id)
        total_route_town_ids.extend(route_town_ids[:-1])  # 마지막 DEPOT 제외
    
    total_route_town_ids.append('DEPOT')  # 최종 DEPOT 추가
    return total_route_town_ids

def main():
    try:
        # 데이터 읽기
        data_df, depot_data, towns_data, locations, demands = read_data('data.csv')
        
        # 데이터 모델 생성
        data = create_data_model(locations, demands)
        
        # 4) 멀티 스타트: 여러 번 실행 -> 최고 해 선택
        # 시간은 예시로 300초씩(num_runs=5번) = 총 25분 정도
        routes, best_dist = solve_vrp_multi_start(
            data, 
            time_limit_seconds=600,
            num_runs=10
        )
        
        # 최적 해 출력/저장
        if routes:
            print("\n=== 산타의 배달 경로 ===")
            for i, route in enumerate(routes, 1):
                route_town_ids = []
                for node_idx in route:
                    if node_idx == 0:
                        route_town_ids.append('DEPOT')
                    else:
                        town_id = towns_data.iloc[node_idx-1]['point_id']
                        route_town_ids.append(town_id)
                print(f"Route #{i}: {' -> '.join(route_town_ids)}")
            
            print(f"\n총 이동 거리(환산): {best_dist:.2f} km")
            
            # 제출 파일 생성
            total_route_town_ids = convert_routes_to_submission(routes, towns_data)
            result_df = pd.DataFrame({'point_id': total_route_town_ids})
            result_df.to_csv('submission_multi_start_2.csv', index=False)
            print("\n[INFO] 제출 파일이 생성되었습니다: submission_multi_start_2.csv")
            
        else:
            print("[ERROR] 해를 찾을 수 없습니다!")
            
    except Exception as e:
        print(f"[ERROR] {e}")

if __name__ == "__main__":
    main()


[INFO] Multi-Start Run #1: FS=10, MH=4, Seed=77214
[INFO] Multi-Start Run #2: FS=9, MH=2, Seed=89768
[INFO] Multi-Start Run #3: FS=13, MH=2, Seed=98087
[INFO] Multi-Start Run #4: FS=11, MH=2, Seed=57873
[INFO] Multi-Start Run #5: FS=13, MH=3, Seed=45789
[INFO] Multi-Start Run #6: FS=9, MH=3, Seed=7992
[INFO] Multi-Start Run #7: FS=11, MH=3, Seed=69410
[INFO] Multi-Start Run #8: FS=11, MH=2, Seed=75223
[INFO] Multi-Start Run #9: FS=10, MH=4, Seed=47594
[INFO] Multi-Start Run #10: FS=8, MH=4, Seed=6947
[INFO] Best Distance from 10 runs: 142681.35

=== 산타의 배달 경로 ===
Route #1: DEPOT -> TOWN_46 -> TOWN_02 -> TOWN_04 -> TOWN_67 -> TOWN_60 -> TOWN_17 -> DEPOT
Route #2: DEPOT -> TOWN_08 -> TOWN_72 -> TOWN_66 -> TOWN_71 -> TOWN_53 -> TOWN_61 -> DEPOT
Route #3: DEPOT -> TOWN_25 -> TOWN_75 -> TOWN_52 -> TOWN_59 -> TOWN_70 -> TOWN_51 -> TOWN_47 -> TOWN_07 -> DEPOT
Route #4: DEPOT -> TOWN_43 -> TOWN_64 -> TOWN_44 -> TOWN_05 -> TOWN_03 -> TOWN_74 -> TOWN_01 -> DEPOT
Route #5: DEPOT -> TOWN_23 -> TOW

In [7]:
import math
import pandas as pd

def calculate_route_distance(submission_path: str, data_path: str) -> float:
    """
    제출 파일의 경로에 대한 총 이동 거리 계산
    
    Args:
        submission_path: 제출 파일 경로 ('point_id' 컬럼을 가진 CSV)
        data_path: 원본 데이터 파일 경로 ('point_id', 'x', 'y' 컬럼을 가진 CSV)
        
    Returns:
        float: 총 이동 거리
    """
    # 데이터 로드
    submission_df = pd.read_csv(submission_path)
    data_df = pd.read_csv(data_path)
    
    # point_id를 기준으로 좌표 정보 병합
    route_df = pd.merge(
        submission_df,
        data_df[['point_id', 'x', 'y']],
        on='point_id',
        how='left'
    )
    
    # 총 거리 계산
    total_distance = 0
    for i in range(len(route_df) - 1):
        # 현재 지점과 다음 지점의 좌표
        current = route_df.iloc[i]
        next_point = route_df.iloc[i + 1]
        
        # 유클리드 거리 계산
        distance = math.sqrt(
            (current['x'] - next_point['x'])**2 + 
            (current['y'] - next_point['y'])**2
        )
        total_distance += distance
        
        # 이동 정보 출력
        print(f"{current['point_id']} -> {next_point['point_id']}: {distance:.4f}")
    
    return total_distance

def main():
    # 파일 경로
    submission_path = 'submission_multi_start_2.csv'  # 제출 파일
    data_path = 'data.csv'  # 원본 데이터
    
    try:
        # 총 거리 계산
        total_distance = calculate_route_distance(submission_path, data_path)
        
        print(f"\n=== 경로 거리 계산 결과 ===")
        print(f"총 이동 거리: {total_distance:.4f}")
        
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

DEPOT -> TOWN_46: 72.9452
TOWN_46 -> TOWN_02: 25.4951
TOWN_02 -> TOWN_04: 8.6023
TOWN_04 -> TOWN_67: 10.8167
TOWN_67 -> TOWN_60: 5.6569
TOWN_60 -> TOWN_17: 17.0000
TOWN_17 -> DEPOT: 91.4385
DEPOT -> TOWN_08: 109.0413
TOWN_08 -> TOWN_72: 14.3178
TOWN_72 -> TOWN_66: 18.9737
TOWN_66 -> TOWN_71: 16.2788
TOWN_71 -> TOWN_53: 8.6023
TOWN_53 -> TOWN_61: 15.2315
TOWN_61 -> DEPOT: 100.4988
DEPOT -> TOWN_25: 91.3947
TOWN_25 -> TOWN_75: 10.0499
TOWN_75 -> TOWN_52: 11.0454
TOWN_52 -> TOWN_59: 4.0000
TOWN_59 -> TOWN_70: 8.0623
TOWN_70 -> TOWN_51: 8.6023
TOWN_51 -> TOWN_47: 14.3178
TOWN_47 -> TOWN_07: 1.0000
TOWN_07 -> DEPOT: 86.0523
DEPOT -> TOWN_43: 101.2719
TOWN_43 -> TOWN_64: 2.8284
TOWN_64 -> TOWN_44: 9.0000
TOWN_44 -> TOWN_05: 7.2801
TOWN_05 -> TOWN_03: 7.0711
TOWN_03 -> TOWN_74: 1.4142
TOWN_74 -> TOWN_01: 4.0000
TOWN_01 -> DEPOT: 99.9600
DEPOT -> TOWN_23: 100.8415
TOWN_23 -> TOWN_14: 4.0000
TOWN_14 -> TOWN_20: 1.4142
TOWN_20 -> TOWN_21: 6.0828
TOWN_21 -> TOWN_24: 1.0000
TOWN_24 -> TOWN_22: 9.0

## 파라미터 조정 4

In [5]:
import math
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

def read_data(csv_path: str):
    """데이터를 읽고 전처리하는 함수"""
    data_df = pd.read_csv(csv_path)
    data_df = data_df.sort_values(by='point_id').reset_index(drop=True)
    
    # DEPOT과 마을 데이터 분리
    depot_data = data_df[data_df['point_id'] == 'DEPOT'].iloc[0]
    towns_data = data_df[data_df['point_id'] != 'DEPOT'].reset_index(drop=True)
    
    # OR-Tools 데이터 준비
    locations = [(depot_data['x'], depot_data['y'])] + list(zip(towns_data['x'], towns_data['y']))
    demands = [0] + list(towns_data['demand'])
    
    return data_df, depot_data, towns_data, locations, demands

def initialize_routes_by_distance(data, depot_data, towns_data):
    """거리를 기준으로 초기 경로 생성"""
    # DEPOT으로부터의 거리 계산
    towns_data['depot_distance'] = ((towns_data['x'] - depot_data['x'])**2 + 
                                  (towns_data['y'] - depot_data['y'])**2).apply(math.sqrt)
    
    # 거리순으로 정렬
    towns_data_sorted = towns_data.sort_values('depot_distance')
    
    return towns_data_sorted

def initialize_routes_by_angle(data, depot_data, towns_data):
    """각도를 기준으로 초기 경로 생성"""
    # DEPOT 기준 각도 계산
    towns_data['angle'] = np.arctan2(towns_data['y'] - depot_data['y'],
                                    towns_data['x'] - depot_data['x'])
    
    # 각도순으로 정렬
    towns_data_sorted = towns_data.sort_values('angle')
    
    return towns_data_sorted

def cluster_demands(towns_data, num_clusters=15):
    """수요를 기준으로 마을들을 클러스터링"""
    # 좌표와 수요를 결합하여 클러스터링
    X = towns_data[['x', 'y', 'demand']].values
    
    # K-means 클러스터링 수행
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    towns_data['cluster'] = kmeans.fit_predict(X)
    
    return towns_data.sort_values('cluster')

def create_data_model(locations, demands):
    """OR-Tools 데이터 모델 생성"""
    data = {}
    # 정수 스케일 설정 (1000으로 변경)
    data['distance_matrix'] = [
        [int(round(math.sqrt((x1[0] - x2[0])**2 + (x1[1] - x2[1])**2) * 1000))
         for x2 in locations]
        for x1 in locations
    ]
    data['demands'] = demands
    data['vehicle_capacities'] = [25] * 15  # 15대의 차량
    data['num_vehicles'] = 15
    data['depot'] = 0
    return data

def improve_initial_solution(data, depot_data, towns_data):
    """여러 초기화 방법을 조합하여 최적의 시작점 찾기"""
    # 1. 거리 기반 정렬
    towns_distance = initialize_routes_by_distance(data, depot_data, towns_data)
    
    # 2. 각도 기반 정렬
    towns_angle = initialize_routes_by_angle(data, depot_data, towns_data)
    
    # 3. 수요 기반 클러스터링
    towns_cluster = cluster_demands(towns_data)
    
    # 각 방법으로 초기 해를 생성하고 가장 좋은 것을 선택
    solutions = []
    for towns in [towns_distance, towns_angle, towns_cluster]:
        solution = solve_vrp_single_run(data, towns)
        solutions.append(solution)
    
    return min(solutions, key=lambda x: x[1])  # 가장 짧은 거리의 해 반환

def solve_vrp_single_run(data, 
                        towns_data,
                        time_limit_seconds=600,
                        first_sol_strategy=routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC):
    """단일 VRP 실행"""
    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']),
        data['num_vehicles'],
        data['depot']
    )
    routing = pywrapcp.RoutingModel(manager)
    
    # 거리 콜백 등록
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    distance_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(distance_callback_index)
    
    # 수요 콜백 등록
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]
    
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    
    # 용량 제약 추가
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # slack
        data['vehicle_capacities'],
        True,
        'Capacity'
    )
    
    # 탐색 파라미터 설정
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = first_sol_strategy
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = time_limit_seconds
    search_parameters.guided_local_search_lambda_coefficient = 0.1
    
    # 해 찾기
    solution = routing.SolveWithParameters(search_parameters)
    if not solution:
        return None, None
    
    # 결과 파싱
    routes = []
    total_distance = 0.0
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        if routing.IsEnd(index):
            continue
            
        route = []
        route_distance = 0
        
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(node)
            prev_index = index
            index = solution.Value(routing.NextVar(index))
            if not routing.IsEnd(index):
                next_node = manager.IndexToNode(index)
                route_distance += data['distance_matrix'][node][next_node] / 1000.0
        
        if len(route) > 1:
            route.append(0)  # DEPOT 복귀
            routes.append(route)
            total_distance += route_distance
    
    return routes, total_distance

def convert_routes_to_submission(routes, towns_data):
    """경로를 제출 형식으로 변환"""
    total_route_town_ids = []
    
    for route in routes:
        route_town_ids = []
        for node_idx in route:
            if node_idx == 0:
                route_town_ids.append('DEPOT')
            else:
                town_id = towns_data.iloc[node_idx - 1]['point_id']
                route_town_ids.append(town_id)
        total_route_town_ids.extend(route_town_ids[:-1])
    
    total_route_town_ids.append('DEPOT')
    return total_route_town_ids

def main():
    try:
        # 데이터 읽기
        data_df, depot_data, towns_data, locations, demands = read_data('data.csv')
        
        # 데이터 모델 생성
        data = create_data_model(locations, demands)
        
        # 개선된 초기 해 생성
        routes, best_dist = improve_initial_solution(data, depot_data, towns_data)
        
        if routes:
            print("\n=== 배달 경로 ===")
            for i, route in enumerate(routes, 1):
                route_town_ids = []
                for node_idx in route:
                    if node_idx == 0:
                        route_town_ids.append('DEPOT')
                    else:
                        town_id = towns_data.iloc[node_idx-1]['point_id']
                        route_town_ids.append(town_id)
                print(f"Route #{i}: {' -> '.join(route_town_ids)}")
            
            print(f"\n총 이동 거리: {best_dist:.4f} km")
            
            # 제출 파일 생성
            total_route_town_ids = convert_routes_to_submission(routes, towns_data)
            result_df = pd.DataFrame({'point_id': total_route_town_ids})
            result_df.to_csv('submission_multi_start_3.csv', index=False)
            print("\n[INFO] 제출 파일이 생성되었습니다: submission_multi_start_3.csv")
            
        else:
            print("[ERROR] 해를 찾을 수 없습니다!")
            
    except Exception as e:
        print(f"[ERROR] {e}")

if __name__ == "__main__":
    main()

C:\Users\jisu0\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\jisu0\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(



=== 배달 경로 ===
Route #1: DEPOT -> TOWN_64 -> TOWN_21 -> TOWN_14 -> TOWN_23 -> TOWN_43 -> TOWN_44 -> DEPOT
Route #2: DEPOT -> TOWN_57 -> TOWN_53 -> TOWN_71 -> TOWN_66 -> TOWN_72 -> TOWN_08 -> TOWN_06 -> TOWN_38 -> DEPOT
Route #3: DEPOT -> TOWN_32 -> TOWN_19 -> TOWN_69 -> TOWN_55 -> TOWN_67 -> TOWN_60 -> TOWN_33 -> DEPOT
Route #4: DEPOT -> TOWN_22 -> TOWN_16 -> TOWN_24 -> TOWN_20 -> TOWN_15 -> DEPOT
Route #5: DEPOT -> TOWN_51 -> TOWN_70 -> TOWN_59 -> TOWN_52 -> TOWN_75 -> TOWN_25 -> TOWN_28 -> TOWN_49 -> DEPOT
Route #6: DEPOT -> TOWN_37 -> TOWN_34 -> TOWN_48 -> TOWN_46 -> TOWN_62 -> TOWN_54 -> DEPOT
Route #7: DEPOT -> TOWN_18 -> TOWN_04 -> TOWN_03 -> TOWN_74 -> TOWN_05 -> TOWN_01 -> TOWN_02 -> TOWN_35 -> DEPOT
Route #8: DEPOT -> TOWN_17 -> TOWN_61 -> TOWN_42 -> TOWN_09 -> TOWN_73 -> TOWN_47 -> TOWN_07 -> TOWN_36 -> TOWN_40 -> TOWN_58 -> TOWN_29 -> DEPOT
Route #9: DEPOT -> TOWN_65 -> TOWN_63 -> TOWN_27 -> TOWN_26 -> TOWN_45 -> DEPOT
Route #10: DEPOT -> TOWN_56 -> TOWN_50 -> TOWN_39 -> TOW

In [13]:
import math
import pandas as pd

def calculate_route_distance(submission_path: str, data_path: str) -> float:
    """
    제출 파일의 경로에 대한 총 이동 거리 계산
    
    Args:
        submission_path: 제출 파일 경로 ('point_id' 컬럼을 가진 CSV)
        data_path: 원본 데이터 파일 경로 ('point_id', 'x', 'y' 컬럼을 가진 CSV)
        
    Returns:
        float: 총 이동 거리
    """
    # 데이터 로드
    submission_df = pd.read_csv(submission_path)
    data_df = pd.read_csv(data_path)
    
    # point_id를 기준으로 좌표 정보 병합
    route_df = pd.merge(
        submission_df,
        data_df[['point_id', 'x', 'y']],
        on='point_id',
        how='left'
    )
    
    # 총 거리 계산
    total_distance = 0
    for i in range(len(route_df) - 1):
        # 현재 지점과 다음 지점의 좌표
        current = route_df.iloc[i]
        next_point = route_df.iloc[i + 1]
        
        # 유클리드 거리 계산
        distance = math.sqrt(
            (current['x'] - next_point['x'])**2 + 
            (current['y'] - next_point['y'])**2
        )
        total_distance += distance
        
        # 이동 정보 출력
        print(f"{current['point_id']} -> {next_point['point_id']}: {distance:.4f}")
    
    return total_distance

def main():
    # 파일 경로
    submission_path = 'submission_multi_start_3.csv'  # 제출 파일
    data_path = 'data.csv'  # 원본 데이터
    
    try:
        # 총 거리 계산
        total_distance = calculate_route_distance(submission_path, data_path)
        
        print(f"\n=== 경로 거리 계산 결과 ===")
        print(f"총 이동 거리: {total_distance:.4f}")
        
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

DEPOT -> TOWN_32: 33.3017
TOWN_32 -> TOWN_73: 55.3173
TOWN_73 -> TOWN_09: 6.4031
TOWN_09 -> TOWN_42: 9.0554
TOWN_42 -> TOWN_61: 2.8284
TOWN_61 -> TOWN_17: 9.4340
TOWN_17 -> TOWN_18: 12.6491
TOWN_18 -> TOWN_48: 9.4868
TOWN_48 -> DEPOT: 71.8401
DEPOT -> TOWN_34: 68.4105
TOWN_34 -> TOWN_19: 34.7131
TOWN_19 -> TOWN_69: 8.2462
TOWN_69 -> TOWN_55: 1.0000
TOWN_55 -> TOWN_67: 14.1421
TOWN_67 -> TOWN_60: 5.6569
TOWN_60 -> TOWN_33: 34.6699
TOWN_33 -> DEPOT: 64.1405
DEPOT -> TOWN_46: 72.9452
TOWN_46 -> TOWN_01: 30.0167
TOWN_01 -> TOWN_05: 6.0000
TOWN_05 -> TOWN_74: 7.2111
TOWN_74 -> TOWN_03: 1.4142
TOWN_03 -> TOWN_04: 8.0623
TOWN_04 -> TOWN_02: 8.6023
TOWN_02 -> TOWN_35: 32.8938
TOWN_35 -> DEPOT: 64.8460
DEPOT -> TOWN_47: 86.8332
TOWN_47 -> TOWN_06: 7.2111
TOWN_06 -> TOWN_08: 15.8114
TOWN_08 -> TOWN_72: 14.3178
TOWN_72 -> TOWN_66: 18.9737
TOWN_66 -> TOWN_71: 16.2788
TOWN_71 -> TOWN_53: 8.6023
TOWN_53 -> TOWN_57: 11.0454
TOWN_57 -> DEPOT: 107.7126
DEPOT -> TOWN_56: 40.2244
TOWN_56 -> TOWN_50: 5.38

## 파라미터 조정 5

In [13]:
import math
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import os
os.environ['OMP_NUM_THREADS'] = '1'

def read_data(csv_path: str):
    """데이터를 읽고 전처리하는 함수"""
    data_df = pd.read_csv(csv_path)
    data_df = data_df.sort_values(by='point_id').reset_index(drop=True)
    
    # DEPOT과 마을 데이터 분리
    depot_data = data_df[data_df['point_id'] == 'DEPOT'].iloc[0]
    towns_data = data_df[data_df['point_id'] != 'DEPOT'].reset_index(drop=True)
    
    # OR-Tools 데이터 준비
    locations = [(depot_data['x'], depot_data['y'])] + list(zip(towns_data['x'], towns_data['y']))
    demands = [0] + list(towns_data['demand'])
    
    return data_df, depot_data, towns_data, locations, demands

def initialize_routes_by_distance(data, depot_data, towns_data):
    """거리를 기준으로 초기 경로 생성"""
    # DEPOT으로부터의 거리 계산
    towns_data['depot_distance'] = ((towns_data['x'] - depot_data['x'])**2 + 
                                  (towns_data['y'] - depot_data['y'])**2).apply(math.sqrt)
    
    # 거리순으로 정렬
    towns_data_sorted = towns_data.sort_values('depot_distance')
    
    return towns_data_sorted

def initialize_routes_by_angle(data, depot_data, towns_data):
    """각도를 기준으로 초기 경로 생성"""
    # DEPOT 기준 각도 계산
    towns_data['angle'] = np.arctan2(towns_data['y'] - depot_data['y'],
                                    towns_data['x'] - depot_data['x'])
    
    # 각도순으로 정렬
    towns_data_sorted = towns_data.sort_values('angle')
    
    return towns_data_sorted

def cluster_demands(towns_data, num_clusters=15):
    """수요를 기준으로 마을들을 클러스터링"""
    # 좌표와 수요를 결합하여 클러스터링
    X = towns_data[['x', 'y', 'demand']].values
    
    # K-means 클러스터링 수행
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
    towns_data['cluster'] = kmeans.fit_predict(X)
    
    return towns_data.sort_values('cluster')

def create_data_model(locations, demands):
    """OR-Tools 데이터 모델 생성"""
    data = {}
    # 정수 스케일 설정 (1000으로 변경)
    data['distance_matrix'] = [
        [int(round(math.sqrt((x1[0] - x2[0])**2 + (x1[1] - x2[1])**2) * 1000))
         for x2 in locations]
        for x1 in locations
    ]
    data['demands'] = demands
    data['vehicle_capacities'] = [25] * 20  # 15대의 차량
    data['num_vehicles'] = 20
    data['depot'] = 0
    return data

def improve_initial_solution(data, depot_data, towns_data):
    """여러 초기화 방법을 조합하여 최적의 시작점 찾기"""
    # 1. 거리 기반 정렬
    towns_distance = initialize_routes_by_distance(data, depot_data, towns_data)
    
    # 2. 각도 기반 정렬
    towns_angle = initialize_routes_by_angle(data, depot_data, towns_data)
    
    # 3. 수요 기반 클러스터링
    towns_cluster = cluster_demands(towns_data)
    
    # 각 방법으로 초기 해를 생성하고 가장 좋은 것을 선택
    solutions = []
    for towns in [towns_distance, towns_angle, towns_cluster]:
        solution = solve_vrp_single_run(data, towns)
        solutions.append(solution)
    
    return min(solutions, key=lambda x: x[1])  # 가장 짧은 거리의 해 반환

def solve_vrp_single_run(data, 
                        towns_data,
                        time_limit_seconds=1000,
                        first_sol_strategy=routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC):
    """단일 VRP 실행"""
    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']),
        data['num_vehicles'],
        data['depot']
    )
    routing = pywrapcp.RoutingModel(manager)
    
    # 거리 콜백 등록
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    distance_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(distance_callback_index)
    
    # 수요 콜백 등록
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]
    
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    
    # 용량 제약 추가
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # slack
        data['vehicle_capacities'],
        True,
        'Capacity'
    )
    
    # 탐색 파라미터 설정
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = first_sol_strategy
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = time_limit_seconds
    search_parameters.guided_local_search_lambda_coefficient = 0.1
    
    # 해 찾기
    solution = routing.SolveWithParameters(search_parameters)
    if not solution:
        return None, None
    
    # 결과 파싱
    routes = []
    total_distance = 0.0
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        if routing.IsEnd(index):
            continue
            
        route = []
        route_distance = 0
        
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(node)
            prev_index = index
            index = solution.Value(routing.NextVar(index))
            if not routing.IsEnd(index):
                next_node = manager.IndexToNode(index)
                route_distance += data['distance_matrix'][node][next_node] / 1000.0
        
        if len(route) > 1:
            route.append(0)  # DEPOT 복귀
            routes.append(route)
            total_distance += route_distance
    
    return routes, total_distance

def convert_routes_to_submission(routes, towns_data):
    """경로를 제출 형식으로 변환"""
    total_route_town_ids = []
    
    for route in routes:
        route_town_ids = []
        for node_idx in route:
            if node_idx == 0:
                route_town_ids.append('DEPOT')
            else:
                town_id = towns_data.iloc[node_idx - 1]['point_id']
                route_town_ids.append(town_id)
        total_route_town_ids.extend(route_town_ids[:-1])
    
    total_route_town_ids.append('DEPOT')
    return total_route_town_ids

def main():
    try:
        # 데이터 읽기
        data_df, depot_data, towns_data, locations, demands = read_data('data.csv')
        
        # 데이터 모델 생성
        data = create_data_model(locations, demands)
        
        # 개선된 초기 해 생성
        routes, best_dist = improve_initial_solution(data, depot_data, towns_data)
        
        if routes:
            print("\n=== 배달 경로 ===")
            for i, route in enumerate(routes, 1):
                route_town_ids = []
                for node_idx in route:
                    if node_idx == 0:
                        route_town_ids.append('DEPOT')
                    else:
                        town_id = towns_data.iloc[node_idx-1]['point_id']
                        route_town_ids.append(town_id)
                print(f"Route #{i}: {' -> '.join(route_town_ids)}")
            
            print(f"\n총 이동 거리: {best_dist:.4f} km")
            
            # 제출 파일 생성
            total_route_town_ids = convert_routes_to_submission(routes, towns_data)
            result_df = pd.DataFrame({'point_id': total_route_town_ids})
            result_df.to_csv('submission_multi_start_4.csv', index=False)
            print("\n[INFO] 제출 파일이 생성되었습니다: submission_multi_start_4.csv")
            
        else:
            print("[ERROR] 해를 찾을 수 없습니다!")
            
    except Exception as e:
        print(f"[ERROR] {e}")

if __name__ == "__main__":
    main()

C:\Users\jisu0\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(



=== 배달 경로 ===
Route #1: DEPOT -> TOWN_54 -> TOWN_62 -> TOWN_35 -> TOWN_46 -> TOWN_48 -> TOWN_34 -> TOWN_37 -> DEPOT
Route #2: DEPOT -> TOWN_16 -> TOWN_24 -> TOWN_21 -> TOWN_15 -> TOWN_30 -> DEPOT
Route #3: DEPOT -> TOWN_63 -> TOWN_28 -> TOWN_27 -> TOWN_26 -> TOWN_49 -> TOWN_45 -> DEPOT
Route #4: DEPOT -> TOWN_19 -> TOWN_57 -> TOWN_69 -> TOWN_55 -> TOWN_67 -> TOWN_60 -> TOWN_33 -> DEPOT
Route #5: DEPOT -> TOWN_53 -> TOWN_71 -> TOWN_66 -> TOWN_72 -> TOWN_08 -> TOWN_06 -> TOWN_47 -> TOWN_38 -> DEPOT
Route #6: DEPOT -> TOWN_10 -> TOWN_11 -> DEPOT
Route #7: DEPOT -> TOWN_22 -> TOWN_20 -> TOWN_14 -> TOWN_23 -> TOWN_43 -> TOWN_64 -> DEPOT
Route #8: DEPOT -> TOWN_58 -> TOWN_40 -> TOWN_25 -> TOWN_75 -> TOWN_52 -> TOWN_59 -> TOWN_70 -> TOWN_51 -> TOWN_36 -> TOWN_29 -> DEPOT
Route #9: DEPOT -> TOWN_41 -> TOWN_39 -> TOWN_65 -> TOWN_50 -> TOWN_56 -> DEPOT
Route #10: DEPOT -> TOWN_32 -> TOWN_07 -> TOWN_73 -> TOWN_09 -> TOWN_42 -> TOWN_61 -> TOWN_17 -> TOWN_18 -> DEPOT
Route #11: DEPOT -> TOWN_02 ->

In [14]:
### import math
import pandas as pd

def calculate_route_distance(submission_path: str, data_path: str) -> float:
    """
    제출 파일의 경로에 대한 총 이동 거리 계산
    
    Args:
        submission_path: 제출 파일 경로 ('point_id' 컬럼을 가진 CSV)
        data_path: 원본 데이터 파일 경로 ('point_id', 'x', 'y' 컬럼을 가진 CSV)
        
    Returns:
        float: 총 이동 거리
    """
    # 데이터 로드
    submission_df = pd.read_csv(submission_path)
    data_df = pd.read_csv(data_path)
    
    # point_id를 기준으로 좌표 정보 병합
    route_df = pd.merge(
        submission_df,
        data_df[['point_id', 'x', 'y']],
        on='point_id',
        how='left'
    )
    
    # 총 거리 계산
    total_distance = 0
    for i in range(len(route_df) - 1):
        # 현재 지점과 다음 지점의 좌표
        current = route_df.iloc[i]
        next_point = route_df.iloc[i + 1]
        
        # 유클리드 거리 계산
        distance = math.sqrt(
            (current['x'] - next_point['x'])**2 + 
            (current['y'] - next_point['y'])**2
        )
        total_distance += distance
        
        # 이동 정보 출력
        print(f"{current['point_id']} -> {next_point['point_id']}: {distance:.4f}")
    
    return total_distance

def main():
    # 파일 경로
    submission_path = 'submission_multi_start_4.csv'  # 제출 파일
    data_path = 'data.csv'  # 원본 데이터
    
    try:
        # 총 거리 계산
        total_distance = calculate_route_distance(submission_path, data_path)
        
        print(f"\n=== 경로 거리 계산 결과 ===")
        print(f"총 이동 거리: {total_distance:.4f}")
        
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

DEPOT -> TOWN_54: 53.4509
TOWN_54 -> TOWN_62: 7.2111
TOWN_62 -> TOWN_35: 7.6158
TOWN_35 -> TOWN_46: 8.4853
TOWN_46 -> TOWN_48: 12.8062
TOWN_48 -> TOWN_34: 3.6056
TOWN_34 -> TOWN_37: 28.3196
TOWN_37 -> DEPOT: 40.2244
DEPOT -> TOWN_16: 93.0054
TOWN_16 -> TOWN_24: 7.0000
TOWN_24 -> TOWN_21: 1.0000
TOWN_21 -> TOWN_15: 15.2643
TOWN_15 -> TOWN_30: 66.0303
TOWN_30 -> DEPOT: 22.4722
DEPOT -> TOWN_63: 74.0000
TOWN_63 -> TOWN_28: 15.2971
TOWN_28 -> TOWN_27: 7.6158
TOWN_27 -> TOWN_26: 5.0990
TOWN_26 -> TOWN_49: 18.0278
TOWN_49 -> TOWN_45: 18.6815
TOWN_45 -> DEPOT: 42.1900
DEPOT -> TOWN_19: 102.7278
TOWN_19 -> TOWN_57: 5.0000
TOWN_57 -> TOWN_69: 4.1231
TOWN_69 -> TOWN_55: 1.0000
TOWN_55 -> TOWN_67: 14.1421
TOWN_67 -> TOWN_60: 5.6569
TOWN_60 -> TOWN_33: 34.6699
TOWN_33 -> DEPOT: 64.1405
DEPOT -> TOWN_53: 114.5600
TOWN_53 -> TOWN_71: 8.6023
TOWN_71 -> TOWN_66: 16.2788
TOWN_66 -> TOWN_72: 18.9737
TOWN_72 -> TOWN_08: 14.3178
TOWN_08 -> TOWN_06: 15.8114
TOWN_06 -> TOWN_47: 7.2111
TOWN_47 -> TOWN_38: 37

## 파라미터 조정 6

In [22]:
import math
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import os
os.environ['OMP_NUM_THREADS'] = '1'

def read_data(csv_path: str):
    """데이터를 읽고 전처리하는 함수"""
    data_df = pd.read_csv(csv_path)
    data_df = data_df.sort_values(by='point_id').reset_index(drop=True)
    
    # DEPOT과 마을 데이터 분리
    depot_data = data_df[data_df['point_id'] == 'DEPOT'].iloc[0]
    towns_data = data_df[data_df['point_id'] != 'DEPOT'].reset_index(drop=True)
    
    # OR-Tools 데이터 준비
    locations = [(depot_data['x'], depot_data['y'])] + list(zip(towns_data['x'], towns_data['y']))
    demands = [0] + list(towns_data['demand'])
    
    return data_df, depot_data, towns_data, locations, demands

def initialize_routes_by_distance(data, depot_data, towns_data):
    """거리를 기준으로 초기 경로 생성"""
    # DEPOT으로부터의 거리 계산
    towns_data['depot_distance'] = ((towns_data['x'] - depot_data['x'])**2 + 
                                  (towns_data['y'] - depot_data['y'])**2).apply(math.sqrt)
    
    # 거리순으로 정렬
    towns_data_sorted = towns_data.sort_values('depot_distance')
    
    return towns_data_sorted

def initialize_routes_by_angle(data, depot_data, towns_data):
    """각도를 기준으로 초기 경로 생성"""
    # DEPOT 기준 각도 계산
    towns_data['angle'] = np.arctan2(towns_data['y'] - depot_data['y'],
                                    towns_data['x'] - depot_data['x'])
    
    # 각도순으로 정렬
    towns_data_sorted = towns_data.sort_values('angle')
    
    return towns_data_sorted

def cluster_demands(towns_data, num_clusters=15):
    """수요를 기준으로 마을들을 클러스터링"""
    # 좌표와 수요를 결합하여 클러스터링
    X = towns_data[['x', 'y', 'demand']].values
    
    # K-means 클러스터링 수행
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
    towns_data['cluster'] = kmeans.fit_predict(X)
    
    return towns_data.sort_values('cluster')

def create_data_model(locations, demands):
    """OR-Tools 데이터 모델 생성"""
    data = {}
    # 정수 스케일 설정 (1000으로 변경)
    data['distance_matrix'] = [
        [int(round(math.sqrt((x1[0] - x2[0])**2 + (x1[1] - x2[1])**2) * 10000))
         for x2 in locations]
        for x1 in locations
    ]
    data['demands'] = demands
    data['vehicle_capacities'] = [25] * 20  # 15대의 차량
    data['num_vehicles'] = 20
    data['depot'] = 0
    return data

def improve_initial_solution(data, depot_data, towns_data):
    """여러 초기화 방법을 조합하여 최적의 시작점 찾기"""
    # 1. 거리 기반 정렬
    towns_distance = initialize_routes_by_distance(data, depot_data, towns_data)
    
    # 2. 각도 기반 정렬
    towns_angle = initialize_routes_by_angle(data, depot_data, towns_data)
    
    # 3. 수요 기반 클러스터링
    towns_cluster = cluster_demands(towns_data)
    
    # 각 방법으로 초기 해를 생성하고 가장 좋은 것을 선택
    solutions = []
    for towns in [towns_distance, towns_angle, towns_cluster]:
        solution = solve_vrp_single_run(data, towns)
        solutions.append(solution)
    
    return min(solutions, key=lambda x: x[1])  # 가장 짧은 거리의 해 반환

def solve_vrp_single_run(data, 
                        towns_data,
                        time_limit_seconds=1200,
                        first_sol_strategy=routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC):
    """단일 VRP 실행"""
    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']),
        data['num_vehicles'],
        data['depot']
    )
    routing = pywrapcp.RoutingModel(manager)
    
    # 거리 콜백 등록
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    distance_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(distance_callback_index)
    
    # 수요 콜백 등록
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]
    
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    
    # 용량 제약 추가
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # slack
        data['vehicle_capacities'],
        True,
        'Capacity'
    )
    
    # 탐색 파라미터 설정
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = first_sol_strategy
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = time_limit_seconds
    search_parameters.guided_local_search_lambda_coefficient = 0.1
    
    # 해 찾기
    solution = routing.SolveWithParameters(search_parameters)
    if not solution:
        return None, None
    
    # 결과 파싱
    routes = []
    total_distance = 0.0
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        if routing.IsEnd(index):
            continue
            
        route = []
        route_distance = 0
        
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(node)
            prev_index = index
            index = solution.Value(routing.NextVar(index))
            if not routing.IsEnd(index):
                next_node = manager.IndexToNode(index)
                route_distance += data['distance_matrix'][node][next_node] / 10000.0
        
        if len(route) > 1:
            route.append(0)  # DEPOT 복귀
            routes.append(route)
            total_distance += route_distance
    
    return routes, total_distance

def convert_routes_to_submission(routes, towns_data):
    """경로를 제출 형식으로 변환"""
    total_route_town_ids = []
    
    for route in routes:
        route_town_ids = []
        for node_idx in route:
            if node_idx == 0:
                route_town_ids.append('DEPOT')
            else:
                town_id = towns_data.iloc[node_idx - 1]['point_id']
                route_town_ids.append(town_id)
        total_route_town_ids.extend(route_town_ids[:-1])
    
    total_route_town_ids.append('DEPOT')
    return total_route_town_ids

def main():
    try:
        # 데이터 읽기
        data_df, depot_data, towns_data, locations, demands = read_data('data.csv')
        
        # 데이터 모델 생성
        data = create_data_model(locations, demands)
        
        # 개선된 초기 해 생성
        routes, best_dist = improve_initial_solution(data, depot_data, towns_data)
        
        if routes:
            print("\n=== 배달 경로 ===")
            for i, route in enumerate(routes, 1):
                route_town_ids = []
                for node_idx in route:
                    if node_idx == 0:
                        route_town_ids.append('DEPOT')
                    else:
                        town_id = towns_data.iloc[node_idx-1]['point_id']
                        route_town_ids.append(town_id)
                print(f"Route #{i}: {' -> '.join(route_town_ids)}")
            
            print(f"\n총 이동 거리: {best_dist:.4f} km")
            
            # 제출 파일 생성
            total_route_town_ids = convert_routes_to_submission(routes, towns_data)
            result_df = pd.DataFrame({'point_id': total_route_town_ids})
            result_df.to_csv('submission_multi_start_5.csv', index=False)
            print("\n[INFO] 제출 파일이 생성되었습니다: submission_multi_start_5.csv")
            
        else:
            print("[ERROR] 해를 찾을 수 없습니다!")
            
    except Exception as e:
        print(f"[ERROR] {e}")

if __name__ == "__main__":
    main()

C:\Users\jisu0\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(



=== 배달 경로 ===
Route #1: DEPOT -> TOWN_19 -> TOWN_57 -> TOWN_69 -> TOWN_55 -> TOWN_67 -> TOWN_60 -> TOWN_33 -> DEPOT
Route #2: DEPOT -> TOWN_58 -> TOWN_63 -> TOWN_28 -> TOWN_27 -> TOWN_26 -> TOWN_49 -> TOWN_45 -> DEPOT
Route #3: DEPOT -> TOWN_22 -> TOWN_16 -> TOWN_24 -> TOWN_21 -> TOWN_15 -> DEPOT
Route #4: DEPOT -> TOWN_37 -> TOWN_48 -> TOWN_46 -> TOWN_35 -> TOWN_62 -> TOWN_30 -> DEPOT
Route #5: DEPOT -> TOWN_34 -> TOWN_18 -> TOWN_17 -> TOWN_61 -> TOWN_42 -> TOWN_09 -> TOWN_73 -> TOWN_07 -> DEPOT
Route #6: DEPOT -> TOWN_10 -> TOWN_11 -> DEPOT
Route #7: DEPOT -> TOWN_20 -> TOWN_14 -> TOWN_23 -> TOWN_43 -> TOWN_64 -> TOWN_54 -> DEPOT
Route #8: DEPOT -> TOWN_40 -> TOWN_25 -> TOWN_75 -> TOWN_52 -> TOWN_59 -> TOWN_70 -> TOWN_51 -> TOWN_36 -> TOWN_38 -> TOWN_29 -> DEPOT
Route #9: DEPOT -> TOWN_41 -> TOWN_39 -> TOWN_65 -> TOWN_50 -> TOWN_56 -> DEPOT
Route #10: DEPOT -> TOWN_47 -> TOWN_72 -> TOWN_66 -> TOWN_71 -> TOWN_53 -> TOWN_08 -> TOWN_06 -> TOWN_32 -> DEPOT
Route #11: DEPOT -> TOWN_02 ->

In [23]:
### import math
import pandas as pd

def calculate_route_distance(submission_path: str, data_path: str) -> float:
    """
    제출 파일의 경로에 대한 총 이동 거리 계산
    
    Args:
        submission_path: 제출 파일 경로 ('point_id' 컬럼을 가진 CSV)
        data_path: 원본 데이터 파일 경로 ('point_id', 'x', 'y' 컬럼을 가진 CSV)
        
    Returns:
        float: 총 이동 거리
    """
    # 데이터 로드
    submission_df = pd.read_csv(submission_path)
    data_df = pd.read_csv(data_path)
    
    # point_id를 기준으로 좌표 정보 병합
    route_df = pd.merge(
        submission_df,
        data_df[['point_id', 'x', 'y']],
        on='point_id',
        how='left'
    )
    
    # 총 거리 계산
    total_distance = 0
    for i in range(len(route_df) - 1):
        # 현재 지점과 다음 지점의 좌표
        current = route_df.iloc[i]
        next_point = route_df.iloc[i + 1]
        
        # 유클리드 거리 계산
        distance = math.sqrt(
            (current['x'] - next_point['x'])**2 + 
            (current['y'] - next_point['y'])**2
        )
        total_distance += distance
        
        # 이동 정보 출력
        print(f"{current['point_id']} -> {next_point['point_id']}: {distance:.4f}")
    
    return total_distance

def main():
    # 파일 경로
    submission_path = 'submission_multi_start_5.csv'  # 제출 파일
    data_path = 'data.csv'  # 원본 데이터
    
    try:
        # 총 거리 계산
        total_distance = calculate_route_distance(submission_path, data_path)
        
        print(f"\n=== 경로 거리 계산 결과 ===")
        print(f"총 이동 거리: {total_distance:.4f}")
        
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()

DEPOT -> TOWN_19: 102.7278
TOWN_19 -> TOWN_57: 5.0000
TOWN_57 -> TOWN_69: 4.1231
TOWN_69 -> TOWN_55: 1.0000
TOWN_55 -> TOWN_67: 14.1421
TOWN_67 -> TOWN_60: 5.6569
TOWN_60 -> TOWN_33: 34.6699
TOWN_33 -> DEPOT: 64.1405
DEPOT -> TOWN_58: 37.6431
TOWN_58 -> TOWN_63: 36.4005
TOWN_63 -> TOWN_28: 15.2971
TOWN_28 -> TOWN_27: 7.6158
TOWN_27 -> TOWN_26: 5.0990
TOWN_26 -> TOWN_49: 18.0278
TOWN_49 -> TOWN_45: 18.6815
TOWN_45 -> DEPOT: 42.1900
DEPOT -> TOWN_22: 91.0055
TOWN_22 -> TOWN_16: 2.0000
TOWN_16 -> TOWN_24: 7.0000
TOWN_24 -> TOWN_21: 1.0000
TOWN_21 -> TOWN_15: 15.2643
TOWN_15 -> DEPOT: 87.5728
DEPOT -> TOWN_37: 40.2244
TOWN_37 -> TOWN_48: 31.8277
TOWN_48 -> TOWN_46: 12.8062
TOWN_46 -> TOWN_35: 8.4853
TOWN_35 -> TOWN_62: 7.6158
TOWN_62 -> TOWN_30: 36.7696
TOWN_30 -> DEPOT: 22.4722
DEPOT -> TOWN_34: 68.4105
TOWN_34 -> TOWN_18: 12.5300
TOWN_18 -> TOWN_17: 12.6491
TOWN_17 -> TOWN_61: 9.4340
TOWN_61 -> TOWN_42: 2.8284
TOWN_42 -> TOWN_09: 9.0554
TOWN_09 -> TOWN_73: 6.4031
TOWN_73 -> TOWN_07: 4.00

## 파라이터 조정 6

In [ ]:
import math
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import os
os.environ['OMP_NUM_THREADS'] = '1'

def read_data(csv_path: str):
    """데이터를 읽고 전처리하는 함수"""
    data_df = pd.read_csv(csv_path)
    data_df = data_df.sort_values(by='point_id').reset_index(drop=True)
    
    # DEPOT과 마을 데이터 분리
    depot_data = data_df[data_df['point_id'] == 'DEPOT'].iloc[0]
    towns_data = data_df[data_df['point_id'] != 'DEPOT'].reset_index(drop=True)
    
    # OR-Tools 데이터 준비
    locations = [(depot_data['x'], depot_data['y'])] + list(zip(towns_data['x'], towns_data['y']))
    demands = [0] + list(towns_data['demand'])
    
    return data_df, depot_data, towns_data, locations, demands

def initialize_routes_by_distance(data, depot_data, towns_data):
    """거리를 기준으로 초기 경로 생성"""
    # DEPOT으로부터의 거리 계산
    towns_data['depot_distance'] = ((towns_data['x'] - depot_data['x'])**2 + 
                                  (towns_data['y'] - depot_data['y'])**2).apply(math.sqrt)
    
    # 거리순으로 정렬
    towns_data_sorted = towns_data.sort_values('depot_distance')
    
    return towns_data_sorted

def initialize_routes_by_angle(data, depot_data, towns_data):
    """각도를 기준으로 초기 경로 생성"""
    # DEPOT 기준 각도 계산
    towns_data['angle'] = np.arctan2(towns_data['y'] - depot_data['y'],
                                    towns_data['x'] - depot_data['x'])
    
    # 각도순으로 정렬
    towns_data_sorted = towns_data.sort_values('angle')
    
    return towns_data_sorted

def cluster_demands(towns_data, num_clusters=15):
    """수요를 기준으로 마을들을 클러스터링"""
    # 좌표와 수요를 결합하여 클러스터링
    X = towns_data[['x', 'y', 'demand']].values
    
    # K-means 클러스터링 수행
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
    towns_data['cluster'] = kmeans.fit_predict(X)
    
    return towns_data.sort_values('cluster')

def create_data_model(locations, demands):
    """OR-Tools 데이터 모델 생성"""
    data = {}
    # 정수 스케일 설정 (1000으로 변경)
    data['distance_matrix'] = [
        [int(round(math.sqrt((x1[0] - x2[0])**2 + (x1[1] - x2[1])**2) * 10000))
         for x2 in locations]
        for x1 in locations
    ]
    data['demands'] = demands
    data['vehicle_capacities'] = [25] * 25  # 15대의 차량
    data['num_vehicles'] = 25
    data['depot'] = 0
    return data

def improve_initial_solution(data, depot_data, towns_data):
    """여러 초기화 방법을 조합하여 최적의 시작점 찾기"""
    # 1. 거리 기반 정렬
    towns_distance = initialize_routes_by_distance(data, depot_data, towns_data)
    
    # 2. 각도 기반 정렬
    towns_angle = initialize_routes_by_angle(data, depot_data, towns_data)
    
    # 3. 수요 기반 클러스터링
    towns_cluster = cluster_demands(towns_data)
    
    # 각 방법으로 초기 해를 생성하고 가장 좋은 것을 선택
    solutions = []
    for towns in [towns_distance, towns_angle, towns_cluster]:
        solution = solve_vrp_single_run(data, towns)
        solutions.append(solution)
    
    return min(solutions, key=lambda x: x[1])  # 가장 짧은 거리의 해 반환

def solve_vrp_single_run(data, 
                        towns_data,
                        time_limit_seconds=1200,
                        first_sol_strategy=routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC):
    """단일 VRP 실행"""
    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']),
        data['num_vehicles'],
        data['depot']
    )
    routing = pywrapcp.RoutingModel(manager)
    
    # 거리 콜백 등록
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    distance_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(distance_callback_index)
    
    # 수요 콜백 등록
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]
    
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    
    # 용량 제약 추가
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # slack
        data['vehicle_capacities'],
        True,
        'Capacity'
    )
    
    # 탐색 파라미터 설정
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = first_sol_strategy
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = time_limit_seconds
    search_parameters.guided_local_search_lambda_coefficient = 0.1
    
    # 해 찾기
    solution = routing.SolveWithParameters(search_parameters)
    if not solution:
        return None, None
    
    # 결과 파싱
    routes = []
    total_distance = 0.0
    
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        if routing.IsEnd(index):
            continue
            
        route = []
        route_distance = 0
        
        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(node)
            prev_index = index
            index = solution.Value(routing.NextVar(index))
            if not routing.IsEnd(index):
                next_node = manager.IndexToNode(index)
                route_distance += data['distance_matrix'][node][next_node] / 10000.0
        
        if len(route) > 1:
            route.append(0)  # DEPOT 복귀
            routes.append(route)
            total_distance += route_distance
    
    return routes, total_distance

def convert_routes_to_submission(routes, towns_data):
    """경로를 제출 형식으로 변환"""
    total_route_town_ids = []
    
    for route in routes:
        route_town_ids = []
        for node_idx in route:
            if node_idx == 0:
                route_town_ids.append('DEPOT')
            else:
                town_id = towns_data.iloc[node_idx - 1]['point_id']
                route_town_ids.append(town_id)
        total_route_town_ids.extend(route_town_ids[:-1])
    
    total_route_town_ids.append('DEPOT')
    return total_route_town_ids

def main():
    try:
        # 데이터 읽기
        data_df, depot_data, towns_data, locations, demands = read_data('data.csv')
        
        # 데이터 모델 생성
        data = create_data_model(locations, demands)
        
        # 개선된 초기 해 생성
        routes, best_dist = improve_initial_solution(data, depot_data, towns_data)
        
        if routes:
            print("\n=== 배달 경로 ===")
            for i, route in enumerate(routes, 1):
                route_town_ids = []
                for node_idx in route:
                    if node_idx == 0:
                        route_town_ids.append('DEPOT')
                    else:
                        town_id = towns_data.iloc[node_idx-1]['point_id']
                        route_town_ids.append(town_id)
                print(f"Route #{i}: {' -> '.join(route_town_ids)}")
            
            print(f"\n총 이동 거리: {best_dist:.4f} km")
            
            # 제출 파일 생성
            total_route_town_ids = convert_routes_to_submission(routes, towns_data)
            result_df = pd.DataFrame({'point_id': total_route_town_ids})
            result_df.to_csv('submission_multi_start_6.csv', index=False)
            print("\n[INFO] 제출 파일이 생성되었습니다: submission_multi_start_6.csv")
            
        else:
            print("[ERROR] 해를 찾을 수 없습니다!")
            
    except Exception as e:
        print(f"[ERROR] {e}")

if __name__ == "__main__":
    main()

C:\Users\jisu0\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [ ]:
### import math
import pandas as pd

def calculate_route_distance(submission_path: str, data_path: str) -> float:
    """
    제출 파일의 경로에 대한 총 이동 거리 계산
    
    Args:
        submission_path: 제출 파일 경로 ('point_id' 컬럼을 가진 CSV)
        data_path: 원본 데이터 파일 경로 ('point_id', 'x', 'y' 컬럼을 가진 CSV)
        
    Returns:
        float: 총 이동 거리
    """
    # 데이터 로드
    submission_df = pd.read_csv(submission_path)
    data_df = pd.read_csv(data_path)
    
    # point_id를 기준으로 좌표 정보 병합
    route_df = pd.merge(
        submission_df,
        data_df[['point_id', 'x', 'y']],
        on='point_id',
        how='left'
    )
    
    # 총 거리 계산
    total_distance = 0
    for i in range(len(route_df) - 1):
        # 현재 지점과 다음 지점의 좌표
        current = route_df.iloc[i]
        next_point = route_df.iloc[i + 1]
        
        # 유클리드 거리 계산
        distance = math.sqrt(
            (current['x'] - next_point['x'])**2 + 
            (current['y'] - next_point['y'])**2
        )
        total_distance += distance
        
        # 이동 정보 출력
        print(f"{current['point_id']} -> {next_point['point_id']}: {distance:.4f}")
    
    return total_distance

def main():
    # 파일 경로
    submission_path = 'submission_multi_start_6.csv'  # 제출 파일
    data_path = 'data.csv'  # 원본 데이터
    
    try:
        # 총 거리 계산
        total_distance = calculate_route_distance(submission_path, data_path)
        
        print(f"\n=== 경로 거리 계산 결과 ===")
        print(f"총 이동 거리: {total_distance:.4f}")
        
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()